Requirements
- 1)a -- demean w1, b -- demean w2; 
- 2)a -- 对于之前生成的csv文件100行 取使得n of inequality为max的beta值; b -- 100行的average (n of inequality 也是average); CI 用同样的方式(即如果beta是 arg max 那CI也是用arg max， 如果beta是mean, CI 也是mean);
- 3）bound：+-100;+-500;+-1000

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
import time
np.set_printoptions(suppress = True)

# Demean

df_w1
- var1 = Relationship_Dum(demean)
- var2 = mi_to_zcta5(demean)
- var3 = FinTechIndicator(demean) * rating_avg(demean)
- var4 = FinTechIndicator(demean) * minority_yelp(demean)


df_w2
- var1 = Relationship_Dum
- var2 = mi_to_zcta5
- var3 = FinTechIndicator(demean) * rating_avg(demean)
- var4 = FinTechIndicator(demean) * minority_yelp(demean)

In [9]:
def Exchange_pairs(df_matched, df_unmatched):
    # t1 = time.time()
    
    df_matched_column = df_matched.columns
    df_matched.columns = df_matched_column + '1m'
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id', 'lender_id1m')
    df1 = pd.merge(df_matched, df_unmatched, on = 'lender_id1m', how = 'inner')
    
    l = df1.columns[:-5].append([df1.columns[-5:] + '1um'])
    df1.columns = l
    df_matched.columns = df_matched_column
    df2 = pd.merge(df_matched, df1, left_on = 'loan_id', right_on = 'loan_id1um', how = 'inner') 
    
    ll = (df2.columns[:6]+'2m').append(df2.columns[6:])
    df2.columns = ll
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id1m', 'lender_id')
    df3 = pd.merge(df_unmatched, df2, left_on = ['lender_id','loan_id'], right_on = ['lender_id2m','loan_id1m'], how = 'inner')
    lll = (df3.columns[:6]+'2um').append(df3.columns[6:])
    df3.columns = lll
    
    df_keep = pd.DataFrame()
    for i in range(1, 5):
        name = "value" + str(i)
        df_keep[name] = df3["var"+str(i)+"1m"] + df3["var"+str(i)+"2m"] - df3["var"+str(i)+"1um"] - df3["var"+str(i)+"2um"]
    # t2 = time.time()
    # print("Running time: ", t2-t1)
    return df_keep

In [30]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_matched = pd.read_stata('sample_matched_NY_demean1.dta').iloc[:,: 7]
df_unmatched = pd.read_stata('sample_unmatched_NY_demean1.dta').iloc[:,: 7]

df_sample = df_w1
# df_sample = df_w2

df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
    
df_keep = Exchange_pairs(df_matched, df_unmatched)

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [ ]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_matched = pd.read_stata('sample_matched_NY_demean1.dta').iloc[:,: 7]
df_unmatched = pd.read_stata('sample_unmatched_NY_demean1.dta').iloc[:,: 7]

df_sample = df_w1
'''
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]

df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2021].iloc[:,: 6]

df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2021].iloc[:,: 6]



df_exchange_pairs = Exchange_pairs(df_matched, df_unmatched)
'''
def Fox_func(num, ratio):
    df_sample = df_w2
    df_matched = df_sample[df_sample["match"] == 1]
    df_unmatched = df_sample[df_sample["match"] == 0]
    df_matched.drop(["match"], axis = 1, inplace = True) 
    df_unmatched.drop(["match"], axis = 1, inplace = True) 
    df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
    df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
    df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
    df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
    
    df_keep = Exchange_pairs(df_matched, df_unmatched)
    
    # df_keep.drop(["value3"], axis = 1, inplace = True) 
    # df_keep.drop(["value4"], axis = 1, inplace = True) 
    
    # df_keep.drop(["value2"], axis = 1) 
    # df_keep.drop(["value3"], axis = 1) 

    def objectfunc(beta, df = df_keep):
        return -sum(df.dot(beta) >=0 )
    t1 = time.time()
    # bounds = [(1, 1.0000000001), (-100, 100), (-100, 100), (-100, 100)]
    # bounds = [(-1.000000001, -1), (-500, 500), (-500, 500)]
    bounds = [(-1000, 1000), (-1000, -1000), (-1000, 1000), (-1000, 1000)]
    # bounds = [(-500, 500), (-500, 500), (-500, 500)]
    # bounds = [(-100, 100), (-100, -100), (-100, 100), (-100, 100)]
    # bounds = [(-500, 500), (-500, -500), (-500, 500), (-500, 500)]
    result = differential_evolution(objectfunc, bounds)
    # print(result)

    r = np.append(result.x, round(-result.fun))
    
    # t2 = time.time()
    # print("Differential Evolution time: ", t2 - t1)
    # print("The number of inequalities satisfied is")
    # print(round(-result.fun))
    # Simulation(num, ratio, df_matched, df_unmatched)
    return r




In [5]:
df_result = pd.DataFrame()
for i in range(0, 100):
    print(i)
    result = Fox_func(100, 0.1)
    df_result = df_result.append([list(result)], ignore_index = True)
df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]

0


/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [14]:
# Don't use argmax or idxmax since if the maximum is achieved in multiple locations, only the first is returned.
num_mean = df_result.mean()
num_max = df_result["Number of of inequalities satisfied"].max()
df_result[df_result["Number of of inequalities satisfied"] == num_max]
df_test = df_result
df_test = df_test.append(df_result[df_result["Number of of inequalities satisfied"] == num_max])
df_test = df_test.append(df_result.mean(axis = 0).rename("mean"))
df_test.to_csv("NY2020;D2;2;0131.csv")

In [163]:
# num is the # of random samples(or the # of simulations)
# ratio is the random sampling ratio, which is 10% here
def Simulation(num, ratio, df_matched, df_unmatched):
    df_result = pd.DataFrame()
    for i in range(0, num):
        # df_matched_sample = Subsample(df_matched, ratio)
        # df_unmatched_sample = Subsample(df_unmatched, ratio)
        
        loan_id = df_unmatched["loan_id"].unique()
        sample_loan_id = np.random.choice(loan_id, round(loan_id.shape[0] * ratio), replace = False)
        df_unmatched_sample = df_unmatched.loc[df_unmatched["loan_id"].isin(sample_loan_id)]
        df_matched_sample = df_matched.loc[df_matched["loan_id"].isin(sample_loan_id)]
       
        df_keep = Exchange_pairs(df_matched_sample, df_unmatched_sample)
        # bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)] # fix beta_1 = 1
        # bounds = [(1, 1.0000000001), (-100, 100), (-100, 100), (-100, 100)]
        # bounds = [(-1.000000001, -1), (-500, 500), (-500, 500)]
        # bounds = [(-1000, 1000), (-1000, -1000), (-1000, 1000), (-1000, 1000)]
        # bounds = [(-500, 500), (-500, 500), (-500, 500)]
        bounds = [(-100, 100), (-100, -100), (-100, 100), (-100, 100)]
        # bounds = [(-500, 500), (-500, -500), (-500, 500), (-500, 500)]
        def objectfunc(beta, df = df_keep):
            return -sum(df.dot(beta) >=0 )
        result = differential_evolution(objectfunc, bounds)
        
        r = np.append(result.x, round(-result.fun))
        
        df_result = df_result.append([list(r)], ignore_index = True)
        # print(i)
    # print("for loop finished")
    df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]
    
    num_mean = df_result.mean()
    num_max = df_result["Number of of inequalities satisfied"].max()
    df_result[df_result["Number of of inequalities satisfied"] == num_max]
    # df_test = df_result
    df_test = pd.DataFrame()
    df_test = df_test.append(df_result[df_result["Number of of inequalities satisfied"] == num_max])
    df_test = df_test.append(df_result.mean(axis = 0).rename("mean"))
    
    df_test.index = pd.Series(["argmax", "mean"])
    
    '''
    print("The 5% quantile of parameters are")
    print(df_result.quantile(0.05))
    print("The 95% quantile of parameters are")
    print(df_result.quantile(0.95))
    '''
    
    return df_test



In [169]:
t1 = time.time()
df_simulation = pd.DataFrame()

for j in range(0, 1000):
    if j%10 == 0:
        print(j)
    df_result = Simulation(100, 0.1, df_matched, df_unmatched)
    df_simulation = df_simulation.append(df_result)

t2 = time.time()
print("Simulation time: ", t2-t1)


0


/opt/conda/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
Simulation time:  99078.253469944


In [172]:
df_simulation.to_csv("Simulation_temp.csv")

In [64]:
df_simulation.to_csv("NY2020;Simulation;D1;2;0219.csv")

In [68]:
df_simulation.loc["argmax"]

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
argmax,407.494049,-500.0,-481.278804,431.991336,66074.0
argmax,433.478797,-500.0,-455.741346,370.116173,66072.0
argmax,341.865634,-500.0,-464.217461,126.582139,65945.0
argmax,380.356649,-500.0,375.071880,499.294645,66190.0
argmax,468.700710,-500.0,-470.382049,436.787175,67402.0
...,...,...,...,...,...
argmax,494.607363,-500.0,31.812032,373.479749,65219.0
argmax,422.333270,-500.0,454.090305,259.944355,63721.0
argmax,376.178505,-500.0,26.138031,452.509809,62692.0
argmax,456.229454,-500.0,-420.591521,497.536896,67889.0


1)N of inequ这里，排名，取第25名，然后找到对应的beta。
2)对于每个beta分开取的，直接用quantile

In [190]:
df_output = pd.DataFrame()
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.05))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.95))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.05))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.95))
df_output

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
0.05,57.867757,-100.0,-96.267707,-25.945352,63557.2500
0.95,99.012125,-100.0,85.008570,98.786337,68308.9000
0.05,83.474660,-100.0,-16.523210,61.203847,56470.3545
0.95,88.323430,-100.0,4.207918,74.472616,57536.2455


In [191]:
df_argmax = df_simulation.loc["argmax"]
df_mean = df_simulation.loc["mean"]
df1 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[50]]
df_output = df_output.append(df1)
df2 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[950]]
df_output = df_output.append(df2)
df3 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[50]]
df_output = df_output.append(df3)
df4 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[950]]
df_output = df_output.append(df4)

In [195]:
df_simulation.to_csv("NY2020;Simulation;D1;1;0221.csv")
df_output.to_csv("NY2020;Result;D1;1;0221.csv")

In [116]:
print("The 10% quantile of parameters are")
print(df_simulation.loc["argmax"].quantile(0.10))
print("The 90% quantile of parameters are")
print(df_simulation.loc["argmax"].quantile(0.90))

The 10% quantile of parameters are
Beta_1                                   358.420266
Beta_2                                  -500.000000
Beta_3                                  -468.969329
Beta_4                                   153.651074
Number of of inequalities satisfied    63875.300000
Name: 0.1, dtype: float64
The 90% quantile of parameters are
Beta_1                                   492.356307
Beta_2                                  -500.000000
Beta_3                                   388.880781
Beta_4                                   485.522528
Number of of inequalities satisfied    67904.900000
Name: 0.9, dtype: float64


In [157]:
a = [1,2,3,4,5,6,7,8,9,10]

In [170]:
np.quantile(a, 0.555)

5.995

In [171]:
df_simulation

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
argmax,88.533565,-100.0,67.189087,94.274684,68182.00
mean,88.167143,-100.0,-1.069554,61.023083,58338.13
argmax,71.742804,-100.0,-38.020566,98.480588,68065.00
mean,88.391785,-100.0,-23.593345,74.715989,56906.53
argmax,36.920264,-100.0,17.899580,97.111893,67303.00
...,...,...,...,...,...
mean,84.162191,-100.0,-10.898381,71.933203,56846.00
argmax,68.705833,-100.0,42.563160,85.908387,66148.00
mean,84.703939,-100.0,-9.097775,69.748535,57301.37
argmax,64.075095,-100.0,-3.299933,86.026190,65119.00
